In [33]:
pwd

data = '../data/previous_application.csv'

f_train, f_test = '../data/application_train.csv', '../data/application_test.csv'
f_train_id, f_test_id = '../data/train_sk_id_curr.txt', '../data/test_sk_id_curr.txt'

import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
pd.set_option("display.max_columns", None)
@np.vectorize
def l1_normalize(val, _max, _min):
  return (val - _min)/(_max - _min + 1e-2)

df_main = pd.read_csv(data, dtype={"SK_ID_CURR":str, "SK_ID_PREV":str})

print(df_main.shape)
print(df_main.head(5))

df_main.dtypes.sort_values()

# add bureau balance
df = df_main.copy()

## Remark

- SK_ID_CURR is the main-key, SK_ID_BUREAU can be agg.

cols_key_all = ["SK_ID_CURR", "SK_ID_PREV"]
cols_key_main = ["SK_ID_CURR"]
cols_ratio = []
# categoricals
cols_cate = [k for k, v in df_main.dtypes.items() if v.__str__() == 'object' and k not in cols_key_all]
# anything else is numerical (regular)
# numerics
cols_numeric = [k for k in df.columns if k not in cols_key_all and k not in cols_cate]



df_proc = df.copy().fillna(dict.fromkeys(cols_numeric, 0))

# one-hot encodings
cols_one_hot = []

for col in tqdm(cols_cate):
  _df_col = pd.get_dummies(df[col], prefix='ft')
  # print(_df_col.head(5))
  for feat_col in _df_col.columns:
    df_proc[feat_col] = _df_col[feat_col]
    cols_one_hot.append(feat_col)

print(f"categorical encodings finished #{len(cols_cate)} columns")

# do aggregation
df_agg = df_proc.groupby(cols_key_main)

numeric_methods = {"mean", "median", "max", "min"}
feature_series = {} 
columns_to_agg = cols_numeric + cols_one_hot + cols_ratio
for col in tqdm(columns_to_agg):
  for method in numeric_methods:
    feature_series[f"{col}_{method}"] = df_agg[col].agg(method)

df_final = pd.DataFrame(feature_series) 
for col in tqdm(df_final.columns):
  _max, _min = df_final[col].max(), df_final[col].min()
  df_final[col] = l1_normalize(df_final[col], _max, _min)



df_final

df_final.to_csv("../data/proc/ft_prev_app.csv")

!head -5  ../data/proc/ft_bureau.csv | tail -4